# Syrah Tasting - Data Cleansing

In [1]:
# the following notebook cleanses data from the MBS2A-TU14 Syrah Tasting performed on 29 January 2021 
# 2 February 2021 - SKacur

In [2]:
# load packages
import numpy as np  # array maniuplation
import pandas as pd  # dataframe mangagement
from scipy import stats  # to assist with statistical analysis
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
import matplotlib.pyplot as plt  # visualization
import seaborn as sns  # visual aesthetics
import ipystata  # to call stata from the notebook
%matplotlib inline

IPyStata is loaded in batch mode.


In [3]:
# load data
Nose_df = pd.read_csv('Syrah_Nose.csv')  # name and read in the Syrah_Nose csv
Palate_df = pd.read_csv('Syrah_Palate.csv')
Nose_df.head()  # to display the first 5 rows of data

,Taster,Wine,Country,Price,Ethanol,Gluc_Fruc,TA,pH,AV,Malic_Acid,...,tree_moss,truffle,mushroom,Wine_Faults,Cork_Taint,Reduction,Heat,VA,Sulfites,Brett
0,1,792,Morocco,15.50€,13.44,0.5,3.28,3.66,0.44,0.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,580,New Zealand,21.70€,12.67,0.5,3.70,3.62,0.61,0.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,189,Thailand,23.30€,12.49,0.0,3.57,3.77,0.59,0.0,...,NaN,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN
3,1,821,Italy,32.00€,13.75,0.0,3.40,3.69,0.54,0.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,961,France,40.00€,12.97,0.5,2.99,3.79,0.55,0.0,...,NaN,NaN,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Clean the Data

In [4]:
Nose_df.info()  # first we need to check the things that need to be cleaned

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Taster           152 non-null    int64  
 1   Wine             152 non-null    int64  
 2   Country          152 non-null    object 
 3   Price            152 non-null    object 
 4   Ethanol          152 non-null    float64
 5   Gluc_Fruc        152 non-null    float64
 6   TA               152 non-null    float64
 7   pH               152 non-null    float64
 8   AV               152 non-null    float64
 9   Malic_Acid       152 non-null    float64
 10  Lactic_Acid      152 non-null    float64
 11  IPT              152 non-null    int64  
 12  Density          152 non-null    float64
 13  Rank             150 non-null    float64
 14  Country_Guess    152 non-null    object 
 15  Correct_Guess    80 non-null     object 
 16  Nose_Count       152 non-null    int64  
 17  Fruity          

In [5]:
# let's start by filling  in the Country_Guess column
# we can use np.where to compare values between two columns and specify what string to put into a third column
Nose_df['Correct_Guess'] = np.where(Nose_df['Country'] == Nose_df['Country_Guess'], 'True', 'False')
Palate_df['Correct_Guess'] = np.where(Palate_df['Country'] == Palate_df['Country_Guess'], 'True', 'False')
Nose_df['Correct_Guess'].head()  # to check our work

0    False
1     True
2    False
3     True
4     True
Name: Correct_Guess, dtype: object

In [6]:
# how many correct guesses were there?
correct = Nose_df.Correct_Guess.value_counts()['True']  # name variable correct based on all conditions where True
incorrect = Nose_df.Correct_Guess.value_counts()['False']
print(correct/incorrect*100)  # divide correct by incorrect and print

40.74074074074074


In [7]:
# now let's replace all Nan values with 0, this will help our future statistical tests to run smoothly
Nose_df = Nose_df.replace(np.nan, 0)  # use numpy replace and specify to exchane Nan with 0
Palate_df = Palate_df.replace(np.nan, 0)  # in order to save the change we have to rename the dataframe
Nose_df.head()                            # use the same name in order to save memory

,Taster,Wine,Country,Price,Ethanol,Gluc_Fruc,TA,pH,AV,Malic_Acid,...,tree_moss,truffle,mushroom,Wine_Faults,Cork_Taint,Reduction,Heat,VA,Sulfites,Brett
0,1,792,Morocco,15.50€,13.44,0.5,3.28,3.66,0.44,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,580,New Zealand,21.70€,12.67,0.5,3.70,3.62,0.61,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,189,Thailand,23.30€,12.49,0.0,3.57,3.77,0.59,0.0,...,0.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0
3,1,821,Italy,32.00€,13.75,0.0,3.40,3.69,0.54,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,961,France,40.00€,12.97,0.5,2.99,3.79,0.55,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# we actually want the Rank column to contain Nan because a 0 will throw off the rankings
Nose_df['Rank'] = Nose_df['Rank'].replace(0, np.nan)
Palate_df['Rank'] = Palate_df['Rank'].replace(0, np.nan) 

In [9]:
Nose_df.info()  # to see what work we have left

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Taster           152 non-null    int64  
 1   Wine             152 non-null    int64  
 2   Country          152 non-null    object 
 3   Price            152 non-null    object 
 4   Ethanol          152 non-null    float64
 5   Gluc_Fruc        152 non-null    float64
 6   TA               152 non-null    float64
 7   pH               152 non-null    float64
 8   AV               152 non-null    float64
 9   Malic_Acid       152 non-null    float64
 10  Lactic_Acid      152 non-null    float64
 11  IPT              152 non-null    int64  
 12  Density          152 non-null    float64
 13  Rank             150 non-null    float64
 14  Country_Guess    152 non-null    object 
 15  Correct_Guess    152 non-null    object 
 16  Nose_Count       152 non-null    int64  
 17  Fruity          

In [10]:
# in the Price column the euro symbol doesn't allow pandas to recognize the values as integers
# we can use pandas string functions to replace the euro symbol with nothing
Nose_df['Price'] = Nose_df['Price'].str.replace('€', '').astype('float64')  # set type to float to make price an int
Palate_df['Price'] = Palate_df['Price'].str.replace('€','').astype('float64')
Nose_df.head()  

,Taster,Wine,Country,Price,Ethanol,Gluc_Fruc,TA,pH,AV,Malic_Acid,...,tree_moss,truffle,mushroom,Wine_Faults,Cork_Taint,Reduction,Heat,VA,Sulfites,Brett
0,1,792,Morocco,15.5,13.44,0.5,3.28,3.66,0.44,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,580,New Zealand,21.7,12.67,0.5,3.70,3.62,0.61,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,189,Thailand,23.3,12.49,0.0,3.57,3.77,0.59,0.0,...,0.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0
3,1,821,Italy,32.0,13.75,0.0,3.40,3.69,0.54,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,961,France,40.0,12.97,0.5,2.99,3.79,0.55,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# write the cleaned df to excel so that it can be shared - remove the leading and trailing ''' to run
'''Nose_df.to_excel("Syrah_Nose_cleaned.xlsx",
             sheet_name='Nose') 
Palate_df.to_excel("Syrah_Palate_cleaned.xlsx",
                  sheet_name='Palate')'''

'Nose_df.to_excel("Syrah_Nose_cleaned.xlsx",\n             sheet_name=\'Nose\') \nPalate_df.to_excel("Syrah_Palate_cleaned.xlsx",\n                  sheet_name=\'Palate\')'